In [3]:
import pandas as pd
import numpy as np

In [23]:
rnames = ['userid','itemid','rating','timestamp']
rating = pd.read_table(r'ml-100k/u.data',names = rnames,engine = 'python')
rating = rating.set_index(['userid','itemid'],drop=False).drop('timestamp',axis = 1)

In [24]:
rating

,,userid,itemid,rating
userid,itemid,,,
196,242,196,242,3
186,302,186,302,3
22,377,22,377,1
244,51,244,51,2
166,346,166,346,1
...,...,...,...,...
880,476,880,476,3
716,204,716,204,5
276,1090,276,1090,1


In [25]:
rating_pair = rating.drop(['timestamp'], axis = 1)
rating_pair

KeyError: "['timestamp'] not found in axis"

In [26]:
rating_matrix = rating.pivot(index = 'userid',columns = 'itemid',values = 'rating')

In [27]:
unames = ['userid','age','gender','occupation','zip code']
users = pd.read_table(r'ml-100k/u.user',sep = '\|',names = unames, engine = 'python')
users.head()

,userid,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [28]:
inames = ['itemid','movie title','release date','video release date','IMDB URL','unknown','Action','Adventure'
          ,'Animation','Children\'s','Comedy','Crime','Documentrary','Drama','Fatasy','Film-Noir','Horror','Musical'
          ,'Mystery','Romance','Sci-Fi','Thriller','War','Western']
items = pd.read_table(r'ml-100k/u.item',sep = '\|',names = inames,engine = 'python',encoding='latin-1')
items.head()

,itemid,movie title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fatasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [29]:
items_matrix = items[['itemid', 'unknown', 'Action', 'Adventure', 'Animation',
                     'Children\'s', 'Comedy', 'Crime', 'Documentrary', 'Drama',
                     'Fatasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                     'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]
items_matrix.head()

,itemid,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentrary,Drama,Fatasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [30]:
num_items = len(items)
num_users = len(users)
num_observation = np.ma.count(rating_pair)
scale = num_items * num_users
print(num_items, num_users, scale, num_observation)

1682 943 1586126 300000


In [31]:
def calMovies(userid):
    #观影记录
    try:
        views = rating.loc[(userid,slice(None)),:]
    except:
        return []
    movies = views['itemid'].values.tolist()
    return movies
users['movies'] = users['userid'].apply(calMovies)
users = users[['userid', 'movies']]
users

,userid,movies
0,1,"[61, 189, 33, 160, 20, 202, 171, 265, 155, 117..."
1,2,"[292, 251, 50, 314, 297, 290, 312, 281, 13, 28..."
2,3,"[335, 245, 337, 343, 323, 331, 294, 332, 328, ..."
3,4,"[264, 303, 361, 357, 260, 356, 294, 288, 50, 3..."
4,5,"[2, 17, 439, 225, 110, 454, 424, 1, 363, 98, 1..."
...,...,...
938,939,"[931, 106, 258, 1054, 689, 476, 409, 121, 1190..."
939,940,"[193, 568, 14, 205, 272, 655, 315, 66, 873, 28..."
940,941,"[147, 124, 117, 181, 993, 258, 7, 475, 257, 15..."
941,942,"[117, 200, 604, 423, 261, 427, 487, 323, 615, ..."


## One-hot encoding

In [32]:
observed_matrix = users.drop('movies', 1).join(
    pd.get_dummies(pd.DataFrame(
        users.movies.tolist()).stack()).astype(int).sum(level=0))
observed_matrix = observed_matrix.set_index(['userid'],drop=True) 

C:\Users\Tim\AppData\Local\Temp/ipykernel_7988/58843463.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  observed_matrix = users.drop('movies', 1).join(
C:\Users\Tim\AppData\Local\Temp/ipykernel_7988/58843463.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  pd.get_dummies(pd.DataFrame(


In [33]:
observed_matrix

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,1673.0,1674.0,1675.0,1676.0,1677.0,1678.0,1679.0,1680.0,1681.0,1682.0
userid,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
940,0,0,0,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
941,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Creating MCAR dataset

In [34]:
rating_pair_MCAR = rating.sample(int(len(rating_pair)*0.9))
def calMovies_MCAR(userid):
    #观影记录
    try:
        views = rating_pair_MCAR.loc[(userid,slice(None)),:]
    except:
        return []
    movies = views['itemid'].values.tolist()
    return movies
users['movies_MCAR'] = users['userid'].apply(calMovies_MCAR)
users = users[['userid', 'movies', 'movies_MCAR']]
users

,userid,movies,movies_MCAR
0,1,"[61, 189, 33, 160, 20, 202, 171, 265, 155, 117...","[220, 135, 139, 53, 95, 31, 132, 232, 226, 87,..."
1,2,"[292, 251, 50, 314, 297, 290, 312, 281, 13, 28...","[272, 284, 274, 242, 127, 19, 298, 311, 285, 3..."
2,3,"[335, 245, 337, 343, 323, 331, 294, 332, 328, ...","[344, 300, 323, 325, 258, 268, 294, 339, 338, ..."
3,4,"[264, 303, 361, 357, 260, 356, 294, 288, 50, 3...","[356, 303, 329, 258, 210, 260, 357, 11, 264, 3..."
4,5,"[2, 17, 439, 225, 110, 454, 424, 1, 363, 98, 1...","[405, 101, 449, 89, 200, 102, 208, 377, 429, 3..."
...,...,...,...
938,939,"[931, 106, 258, 1054, 689, 476, 409, 121, 1190...","[255, 818, 1190, 993, 471, 1028, 258, 252, 274..."
939,940,"[193, 568, 14, 205, 272, 655, 315, 66, 873, 28...","[171, 294, 313, 628, 70, 173, 191, 708, 56, 42..."
940,941,"[147, 124, 117, 181, 993, 258, 7, 475, 257, 15...","[358, 15, 294, 258, 124, 147, 298, 257, 117, 7..."
941,942,"[117, 200, 604, 423, 261, 427, 487, 323, 615, ...","[892, 423, 347, 258, 604, 183, 1028, 71, 193, ..."


In [35]:
observed_matrix_MCAR = users.drop(['movies','movies_MCAR'], 1).join(
    pd.get_dummies(pd.DataFrame(
        users.movies_MCAR.tolist()).stack()).astype(int).sum(level=0))
observed_matrix_MCAR = observed_matrix_MCAR.set_index(['userid'],drop=True) 
observed_matrix_MCAR

C:\Users\Tim\AppData\Local\Temp/ipykernel_7988/99084375.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  observed_matrix_MCAR = users.drop(['movies','movies_MCAR'], 1).join(
C:\Users\Tim\AppData\Local\Temp/ipykernel_7988/99084375.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  pd.get_dummies(pd.DataFrame(


,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,1673.0,1674.0,1675.0,1676.0,1677.0,1678.0,1679.0,1680.0,1681.0,1682.0
userid,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
940,0,0,0,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
941,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
rating_MCAR_matrix = rating_pair_MCAR.pivot(index = 'userid',columns = 'itemid',values = 'rating')
rating_MCAR_matrix

itemid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
rating_matrix

itemid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Propensity

In [52]:
rating_counts = rating_pair['rating'].value_counts().sort_index()
rating_MCAR_counts = rating_pair_MCAR['rating'].value_counts().sort_index()
rating_counts, rating_MCAR_counts

(1     6110
 2    11370
 3    27145
 4    34174
 5    21201
 Name: rating, dtype: int64,
 1     5474
 2    10221
 3    24478
 4    30795
 5    19032
 Name: rating, dtype: int64)

In [65]:
# P(Oui = 1 | Yui = r) = P(Y=r|O=1)P(O=1)/P(Y=r)
P_observed = observed_matrix.values.sum() / scale
P_observed  # P(O=1)

P_r_observed_list = []  # P(Y=r|O=1)
for i in rating_counts:
    P_r_observed_list.append(i / observed_matrix.values.sum())
P_r_observed_list = np.array(P_r_observed_list)

P_r_list = []  #P(Y=r)
for i in rating_MCAR_counts:
    P_r_list.append(i / observed_matrix_MCAR.values.sum())
P_r_list = np.array(P_r_list)

propensities = P_r_observed_list * P_observed / P_r_list  #propensities of each item

print("P(O=1) =", P_observed)
print("P(Y=r|O=1) =", P_r_observed_list)
print("P(Y=r) =", P_r_list)
print("propensities for each rating: ", propensities)

P(O=1) = 0.06304669364224531
P(Y=r|O=1) = [0.0611  0.1137  0.27145 0.34174 0.21201]
P(Y=r) = [0.06082222 0.11356667 0.27197778 0.34216667 0.21146667]
propensities for each rating:  [0.06333463 0.06312071 0.06292435 0.06296808 0.06320868]


In [66]:
rating

,,userid,itemid,rating,propensities
userid,itemid,,,,
196,242,196,242,3,0.062924
186,302,186,302,3,0.062924
22,377,22,377,1,0.063335
244,51,244,51,2,0.063121
166,346,166,346,1,0.063335
...,...,...,...,...,...
880,476,880,476,3,0.062924
716,204,716,204,5,0.063209
276,1090,276,1090,1,0.063335


In [74]:
rating['propensities'] = rating['rating'].apply (lambda x: propensities[x-1])
rating.head()

,,userid,itemid,rating,propensities
userid,itemid,,,,
196,242,196,242,3,0.062924
186,302,186,302,3,0.062924
22,377,22,377,1,0.063335
244,51,244,51,2,0.063121
166,346,166,346,1,0.063335


In [76]:
propensity_matrix = rating.pivot(index = 'userid',columns = 'itemid',values = 'propensities')

In [77]:
propensity_matrix

itemid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,0.063209,0.062924,0.062968,0.062924,0.062924,0.063209,0.062968,0.063335,0.063209,0.062924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.062968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.062968,0.062924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063209,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,0.063121,NaN,NaN,0.062968,0.063209,0.062924,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,0.063209,NaN,NaN,NaN,NaN,NaN,0.062968,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
